# 1.1 Feature extraction

In this iteration, (iter 1), we want to use machine learning models. For a better classification with machine learning models, we extract more features, and evaluate the extracted features to see what are the best for classification.

### 1. Adding the libraries we need

In [1]:
import os
import skimage as si
from skimage import io, measure
import pandas as pd
import numpy as np

### 2. Load in all images

All images are loaded in into an array as a tuple of (image, label). This way we know what image we are dealing with. We also lowercase all the labels as "Tyr" is in uppercase, while everything else is in lowercase.

In [2]:
directory = "../dataset-images/" # Path to the dataset
images = [] # List of images

for filename in os.listdir(directory):
    # Check if the file is an image
    if filename.endswith(".png"):
        label = filename.split("_")[0].lower() # Get the label
        # Label and load the image
        img = io.imread(os.path.join(directory, filename))
        images.append((img, label))

# Print the number of images
print("Number of images: " + str(len(images)))
print("All labels: " + str(set([label for _, label in images])))

Number of images: 1472
All labels: {'tyr', 'wealth', 'serpent', 'oak', 'spear', 'gift', 'elk-sedge', 'ash', 'bow', 'joy', 'need', 'sun'}


### 3. Feature extraction

In this iteration, we want to extract as many features as possible. We extract 2 types of features:

1. Region prop features where the returned datatype is a positive (for selecting K best features with `chi2` later on) number and not a float (so we can store it in a CSV)
2. Smart custom features

We created a collection of 2 smart features: `vertical and horizontal symmetry` and `feature density for 9 regions of an image`

Some runes are symmetric horizontally, some vertically, some not at all. This provides us a valuable new feature. With feature density, we count the amount of black pixels for each 9 regions (grid) of an image. This tells us in which region does most of the information (black pixel sum) is.

First, we define some helper functions.

In [3]:
def pad_arrays_to_same_shape(arr1, arr2):
    """Make arrays the same same shape

    If `arr1` or `arr2` are not of the same shape, this function will pad the smaller array to match the shape of the bigger one.
    """
    # Get the shapes of the arrays
    shape1 = np.shape(arr1)
    shape2 = np.shape(arr2)

    # If the shapes are not the same, pad the smaller array with zeros
    if shape1 != shape2:
        # Determine which array is smaller
        if shape1[0] < shape2[0]:
            # Pad arr1 with zeros
            arr1 = np.pad(arr1, ((0, shape2[0] - shape1[0]), (0, 0)), 'constant', constant_values=(0))
        elif shape1[0] > shape2[0]:
            # Pad arr2 with zeros
            arr2 = np.pad(arr2, ((0, shape1[0] - shape2[0]), (0, 0)), 'constant', constant_values=(0))

        if shape1[1] < shape2[1]:
            # Pad arr1 with zeros
            arr1 = np.pad(arr1, ((0, 0), (0, shape2[1] - shape1[1])), 'constant', constant_values=(0))
        elif shape1[1] > shape2[1]:
            # Pad arr2 with zeros
            arr2 = np.pad(arr2, ((0, 0), (0, shape1[1] - shape2[1])), 'constant', constant_values=(0))
    
    return arr1, arr2

In [4]:
def crop_img(img):
    """Crop the image to where black pixels can be found

    This function makes sure that each image is cropped down to the rune's dimension.
    """
     # find the first black pixel from the left
    left = 0
    for i in range(img.shape[1]):
        if np.any(img[:, i] == 0):
            left = i
            break
    # find the first black pixel from the right
    right = 0
    for i in reversed(range(img.shape[1])):
        if np.any(img[:, i] == 0):
            right = i
            break
    # find the first black pixel from the top
    top = 0
    for i in range(img.shape[0]):
        if np.any(img[i, :] == 0):
            top = i
            break
    # find the first black pixel from the bottom
    bottom = 0
    for i in reversed(range(img.shape[0])):
        if np.any(img[i, :] == 0):
            bottom = i
            break
    # crop the image to the smallest rectangle containing the image
    cropped_img = img[top:bottom+1, left:right+1]
    return cropped_img

In [5]:
def h_v_symmetry_errors(img_symmetry):
    """Calculate the horizontal and vertical symmetry error rate of an image

    This function returns a tuple of 2 floats: `(horizontal_sym_errors, vertical_sym_errors)`

    Each returned number is a percentage of errors on how many pixels are not symmetric. 1 means the image not symmetric at all (100% errors)
    and 0 means every pixel is symmetric (0% errors)
    """
    middle_horizontal = img_symmetry.shape[0] // 2
    middle_vertical = img_symmetry.shape[1] // 2
    # check if the image is horizontally symmetric
    top = img_symmetry[:middle_horizontal, :]
    bottom = np.flip(img_symmetry[middle_horizontal:, :], axis=0)
    # check if the shape is the same for top and bottom and if not, insert rows of zeros
    top, bottom = pad_arrays_to_same_shape(top, bottom)
    result = top ^ bottom
    sum_result = np.sum(result) / (result.shape[0] * result.shape[1])
    horizontal_sym_errors = sum_result
    # check if the image is vertically symmetric
    left = img_symmetry[:, :middle_vertical]
    right = np.flip(img_symmetry[:, middle_vertical:], axis=1)
    # check if the shape is the same for left and right and if not, insert columns of zeros
    left, right = pad_arrays_to_same_shape(left, right)
    result = left ^ right
    sum_result = np.sum(result) / (result.shape[0] * result.shape[1])
    vertical_sym_errors = sum_result
    
    return horizontal_sym_errors, vertical_sym_errors

In [6]:
def count_black_pixels(img, regions=1):
    """Count the amount of black pixels in an image

    An optional `regions` may be set to count the black pixels for different regions (grids) of the original image

    This function returns an array of each defined grid's (default 1) summed black pixels
    """
    # split the image into a grids
    img_split = np.array_split(img, regions)
    # get plack pixels for each grid
    black_pixels = []
    for i in range(regions):
        for j in range(regions):
            black_pixels.append(np.sum(img_split[i][j] == 0))
    return black_pixels

Now we create a new Pandas dataframe with all our extracted features combined with our smart features. To get the best results, we first crop the image, then resize the image to the original (128x128) size. This makes sure that the image takes up the whole space, and not just a small part of it (in case someone drew a rune in the corner of the image). We also add a 2 pixel border on top of our image. This is because the runes are stretched to the edges of the image, which would result in incorrect hole count (since the image touches the edges, resulting in more holes). This step helps us to get a more accurate hole count.

In [7]:
extracted_features = []

for img, label in images:
    # if the image is not RGB, convert it
    if len(img.shape) == 2:
        img = si.color.gray2rgb(img)
    
    # remove the alpha channel
    img = img[:, :, :3]

    # binary color scale
    img = si.color.rgb2gray(img)
    threshold_value = si.filters.threshold_otsu(img)
    img = img > threshold_value

    # Crop the image to where the runes can be found so we can resize the rune to full size
    img = crop_img(img)

    # Resize the cropped image to 128x128
    img = si.transform.resize(img, (124, 124)) # not 128x128 because of the padding for 2 pixels border we add later

    # Check if the image is horizontally symmetric
    horizontal_sym_errors, vertical_sym_errors = h_v_symmetry_errors(img.copy())

    # Apply erosion
    img = si.morphology.binary_erosion(img, si.morphology.square(3))

    # add a 2 pixel white border around img to get correct hole count
    img = np.pad(img, ((2, 2), (2, 2)), 'constant', constant_values=(1))

    # get black pixel count for 3 x 3 sections of the img
    black_pixels = count_black_pixels(img, 3)

    # Flip colors to get the inverted image so the labeling will work on the drawn rune
    img_inverted = np.invert(img.copy())

    # Label connected components in the binary image
    labeled_image_inverted = measure.label(img_inverted.astype(int))
    labeled_image = measure.label(img.astype(int))

    # Extract features from the labeled regions
    props_inverted = measure.regionprops(labeled_image_inverted)
    props = measure.regionprops(labeled_image)

    # Iterate through labeled regions and extract features from the first region
    if len(props_inverted) > 0: # if there are no regions, that means that the image has no drawing in it, we can thus ignore the else block
        first_region = props_inverted[0]
        features = { # all our extracted features
            # ---- Region prop features ----
            "label": label,
            "area": first_region.area,
            "area_filled": first_region.area_filled,
            "area_convex": first_region.area_convex,
            "axis_major_length": first_region.axis_major_length,
            "axis_minor_length": first_region.axis_minor_length,
            "eccentricity": first_region.eccentricity,
            "equivalent_diameter_area": first_region.equivalent_diameter_area,
            "extent": first_region.extent,
            "feret_diameter_max": first_region.feret_diameter_max,
            "perimeter": first_region.perimeter,
            "solidity": first_region.solidity,
            "holes": len(props)-1,
            # ---- Smart features ----
            "h_sym_err_percent": horizontal_sym_errors,
            "v_sym_err_percent": vertical_sym_errors,
            "pixelsum_tl": black_pixels[0],
            "pixelsum_tm": black_pixels[1],
            "pixelsum_tr": black_pixels[2],
            "pixelsum_ml": black_pixels[3],
            "pixelsum_mm": black_pixels[4],
            "pixelsum_mr": black_pixels[5],
            "pixelsum_bl": black_pixels[6],
            "pixelsum_bm": black_pixels[7],
            "pixelsum_br": black_pixels[8]
        }
        extracted_features.append((features))

features_df = pd.DataFrame(extracted_features)

print(features_df.describe())

               area   area_filled   area_convex  axis_major_length  \
count   1472.000000   1472.000000   1472.000000        1472.000000   
mean    6107.643342   6470.124321  11039.891984         143.646528   
std     1779.865675   2090.570961   1752.918338          16.273829   
min     1469.000000   1469.000000   1808.000000         110.482240   
25%     5013.250000   5085.000000   9668.750000         132.328105   
50%     5990.000000   6143.500000  10519.000000         142.231762   
75%     6853.500000   7304.500000  12155.250000         153.240164   
max    15376.000000  15376.000000  15376.000000         202.711313   

       axis_minor_length  eccentricity  equivalent_diameter_area       extent  \
count        1472.000000   1472.000000               1472.000000  1472.000000   
mean          104.686584      0.639563                 87.404364     0.398024   
std            14.929751      0.168373                 11.707400     0.115475   
min            14.198591      0.000000       

### 4. Save the data

Save our selected best features to a csv file for later use.

In [8]:
directory = "../dataset-numpy/" 
path = os.path.join(directory, '1.1 - features.csv')

features_df.to_csv(path, index=False)